In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import hstack

import time

#### some helper stuff for tracking performance over the duration of the project

In [2]:
def write_model_timestamp(model_type, kfolds, scores, note):
    '''
    Parameters:
    model_type = string description of the model(s) used to make these scores
    kfolds     = how many folds in kfold cross validation used
    scores     = list of ROC AUC avg scores of models for each class, floats should be like 0.9784
    note       = string, whatever is of note about the model, made a change or whatever
    
    Returns:
    None, but writes (appends) a line to scores.txt in the root directory so that progress can be tracked
    The format is:
            time(s)~model_type~kfold~avg_roc_auc~toxic_auc~s_toxic_auc~obscene_auc~threat_auc~insult_auc~i_hate_auc~notes
            
    scores.txt is a tilde '~' seperated CSV like:
        time~model_type~kfold~avg_roc_auc~toxic_auc~s_toxic_auc~obscene_auc~threat_auc~insult_auc~i_hate_auc~notes
        1520303252~0.9794005980274005~note something
    '''

    out_text = "{:10.0f}~{:}~{:2d}~{:0.8f}~{:0.8f}~{:0.8f}~{:0.8f}~{:0.8f}~{:0.8f}~{:0.8f}~{:}\n".format(time.time(), 
                                             model_type, 
                                             kfolds, 
                                             np.mean(scores),
                                             scores[0],
                                             scores[1],
                                             scores[2],
                                             scores[3],
                                             scores[4],
                                             scores[5],                                                
                                             note)
    
    with open("../scores.txt", 'a') as out_file:
        out_file.write(out_text)
        
        print("wrote:")
        print(out_text)
        print("to file")

#### Load data and light processing

In [3]:
train = pd.read_csv('../data/train.csv').fillna(' ')
test = pd.read_csv('../data/test.csv').fillna(' ')

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

#### Vectorize words from both corpuses (corpi?)

In [4]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=50000)    # 10k was initial

word_vectorizer.fit(all_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=50000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=True,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=True,
        vocabulary=None)

In [5]:
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

print("train shape:", train_word_features.shape)
print("test shape:", test_word_features.shape)

train shape: (159571, 50000)
test shape: (153164, 50000)


In [9]:
scores = []

NUM_FOLDS = 10

train_features = train_word_features.copy()

submission = pd.DataFrame.from_dict({'id': test['id']})

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag')

    kfold = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=1337)
    
#     results = cross_val_score(classifier, train_features, train_target, cv=5, n_jobs=-1, scoring='roc_auc')
    results = cross_val_score(classifier, train_features, train_target, cv=kfold, scoring='roc_auc', n_jobs=-1)
    
    print('CV Spread for class "{}":'.format(class_name))
    for result in results:
        print("    {:0.4f}".format(result), end=" ")
        
    print(" ")
        
    cv_score = np.mean(results)
    scores.append(cv_score)
    
    print('    CV score for class "{}" is {:0.4}\n'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

print('Total CV score is {:0.4f}'.format(np.mean(scores)))

# write_model_timestamp('logistic regression', NUM_FOLDS, scores, "first model: word to vec max 100k features")

CV Spread for class "toxic":
    0.9709     0.9694     0.9714     0.9721     0.9732     0.9720     0.9684     0.9716     0.9727     0.9729  
    CV score for class "toxic" is 0.9715

CV Spread for class "severe_toxic":
    0.9838     0.9876     0.9816     0.9862     0.9876     0.9838     0.9883     0.9873     0.9878     0.9889  
    CV score for class "severe_toxic" is 0.9863

CV Spread for class "obscene":
    0.9884     0.9871     0.9842     0.9854     0.9878     0.9831     0.9868     0.9895     0.9876     0.9862  
    CV score for class "obscene" is 0.9866

CV Spread for class "threat":
    0.9723     0.9904     0.9919     0.9851     0.9727     0.9880     0.9897     0.9782     0.9870     0.9718  
    CV score for class "threat" is 0.9827

CV Spread for class "insult":
    0.9790     0.9765     0.9784     0.9754     0.9798     0.9749     0.9762     0.9776     0.9815     0.9790  
    CV score for class "insult" is 0.9778

CV Spread for class "identity_hate":
    0.9782     0.9723     

In [9]:
print("{: <14} {:}".format("CLASS", "AVG ROC AUC"))

for item in zip(class_names, scores):
    print("{: <14} {:0.4f}".format(item[0], item[1]))
    
print("Overall Average ROC AUC: {:0.4f}".format(np.mean(scores)))

CLASS          AVG ROC AUC
toxic          0.9715
severe_toxic   0.9863
obscene        0.9866
threat         0.9827
insult         0.9778
identity_hate  0.9770
Overall Average ROC AUC: 0.9803


In [10]:
submission.to_csv('../submissions/logistic_regression_tfidf_submission.csv.gz', index=False, compression="gzip")

In [11]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.993519,0.120255,0.988003,0.020985,0.860358,0.174008
1,0000247867823ef7,0.009684,0.003237,0.005897,0.001713,0.009233,0.003706
2,00013b17ad220c46,0.035918,0.003873,0.014029,0.001669,0.018623,0.004621
3,00017563c3f7919a,0.003429,0.002087,0.003250,0.001071,0.003966,0.000880
4,00017695ad8997eb,0.039584,0.002397,0.010039,0.001819,0.012766,0.002737


In [12]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153164 entries, 0 to 153163
Data columns (total 7 columns):
id               153164 non-null object
toxic            153164 non-null float64
severe_toxic     153164 non-null float64
obscene          153164 non-null float64
threat           153164 non-null float64
insult           153164 non-null float64
identity_hate    153164 non-null float64
dtypes: float64(6), object(1)
memory usage: 8.2+ MB
